# Example

Taken from nvidia's website https://devblogs.nvidia.com/accelerate-r-applications-cuda/

You need to know how to compile C code on linux. We modified various things to make the example work on our server.

# Compile code

In [9]:
system("cat cufft-R.cu", intern = TRUE)

[1] "#include <R.h>"                                                              
 [2] "#include <cufft.h>"                                                          
 [3] "/* This function is written for R to compute 1D FFT."                        
 [4] "   n - [IN] the number of complex we want to compute"                        
 [5] "   inverse - [IN] set to 1 if use inverse mode"                              
 [6] "   h_idata_re - [IN] input data from host (R, real part)"                    
 [7] "   h_idata_im - [IN] input data from host (R, imaginary part)"               
 [8] "   h_odata_re - [OUT] results (real) allocated by caller"                    
 [9] "   h_odata_im - [OUT] results (imaginary) allocated by caller"               
[10] "*/"                                                                          
[11] "extern \"C\""                                                                
[12] "void cufft(int *n, int *inverse, double *h_idata_re,"                        
[13] "           double *h_idata_im, double *h_odata_re, double *h_odata_im)"      
[14] "{"                                                                           
[15] "  cufftHandle plan;"                                                         
[16] "  cufftDoubleComplex *d_data, *h_data;"                                      
[17] "  cudaMalloc((void**)&d_data, sizeof(cufftDoubleComplex)*(*n));"             
[18] "  h_data = (cufftDoubleComplex *) malloc(sizeof(cufftDoubleComplex) * (*n));"
[19] ""                                                                            
[20] "  // Convert data to cufftDoubleComplex type"                                
[21] "  for(int i=0; i< *n; i++) {"                                                
[22] "    h_data[i].x = h_idata_re[i];"                                            
[23] "    h_data[i].y = h_idata_im[i];"                                            
[24] "  }"                                                                         
[25] " "                                                                           
[26] "  cudaMemcpy(d_data, h_data, sizeof(cufftDoubleComplex) * (*n), "            
[27] "             cudaMemcpyHostToDevice);"                                       
[28] "  // Use the CUFFT plan to transform the signal in place."                   
[29] "  cufftPlan1d(&plan, *n, CUFFT_Z2Z, 1);"                                     
[30] "  if (!*inverse ) {"                                                         
[31] "    cufftExecZ2Z(plan, d_data, d_data, CUFFT_FORWARD);"                      
[32] "  } else {"                                                                  
[33] "    cufftExecZ2Z(plan, d_data, d_data, CUFFT_INVERSE);"                      
[34] "  }"                                                                         
[35] ""                                                                            
[36] "  cudaMemcpy(h_data, d_data, sizeof(cufftDoubleComplex) * (*n), "            
[37] "  cudaMemcpyDeviceToHost);"                                                  
[38] "  // split cufftDoubleComplex to double array"                               
[39] "  for(int i=0; i<*n; i++) {"                                                 
[40] "    h_odata_re[i] = h_data[i].x;"                                            
[41] "    h_odata_im[i] = h_data[i].y;"                                            
[42] "  }"                                                                         
[43] " "                                                                           
[44] "  // Destroy the CUFFT plan and free memory."                                
[45] "  cufftDestroy(plan);"                                                       
[46] "  cudaFree(d_data);"                                                         
[47] "  free(h_data);"                                                             
[48] "}"

In [10]:
system("nvcc -O3 -arch=sm_35 -G -I/usr/local/cuda/include          -I/usr/lib64/R/include           -L/usr/lib64/R/lib -lR           -L/usr/local/cuda/lib64 -lcufft           --shared -Xcompiler -fPIC -o cufft.so cufft-R.cu", intern =  TRUE)

character(0)

In [14]:
dir()

[1] "cufft-R.cu"              "cufft.so"               
[3] "gpu_install_fail1.ipynb" "LICENSE"                
[5] "README.md"               "Untitled.ipynb"

## load shared lib

wrapper function in R

In [15]:
cufft1D <- function(x, inverse=FALSE)
{
  if(!is.loaded("cufft")) {
    dyn.load("cufft.so")
  }
  n <- length(x)
  rst <- .C("cufft",
  as.integer(n),
  as.integer(inverse),
  as.double(Re(z)),
  as.double(Im(z)),
  re=double(length=n),
  im=double(length=n))
  rst <- complex(real = rst[["re"]], imaginary = rst[["im"]])
  return(rst)
}

# Compare running time

In [34]:
num <- 2**20
z <- complex(real = stats::rnorm(num), imaginary = stats::rnorm(num))

system.time( cpu <- fft(z, inverse = T) )

   user  system elapsed 
   0.06    0.00    0.06 

In [35]:
system.time( gpu <- cufft1D(z, inverse = T) )

   user  system elapsed 
  0.043   0.001   0.044 

In [36]:
system.time( gpu <- fft(z) )

   user  system elapsed 
  0.063   0.001   0.062 

In [37]:
system.time( gpu <- cufft1D(z) )

   user  system elapsed 
  0.046   0.003   0.049 